<a href="https://colab.research.google.com/github/notnsas/data-science-capstone-project-college/blob/main/models/lstm_crf_absa_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datasets import load_dataset
dataset = load_dataset(
    "json",
    data_files={
        "train": r"D:\Code\Capstone\dataset\absa_train.json",
        "test":  r"D:\Code\Capstone\dataset\absa_test.json"
    }
)

D:\anaconda3\envs\capstone\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def find_sequence(lst, seq):
    for i in range(len(lst) - len(seq) + 1):
        match = True
        for j in range(len(seq)):
            if seq[j] not in lst[i + j]:
                match = False
                break
        if match:
            return i, i + len(seq)
    return -1
def labeling_ner(batch):
    labels = []
    batch = batch
    tokens = batch['token']
    # print(f'token : {tokens}')
    # # for i, tokens in enumerate(batch['token']):
    # print("\n" + "-"*60)
    # print(f"🔹 Sample {i}")
    # print("-"*60)

    # Create default label 0 (O-tag)
    label = [0] * len(tokens)

    # Mark aspect terms with proper label IDs
    # aspects = batch["aspects"][i]
    aspects = batch["aspects"]
    for aspect in aspects:
        # start_id, end_id = aspect['from'], aspect['to']
        t = aspect
        # print(f'aspect : {t}')
        term = aspect["term"].split()
        # print(f'term : {term}')
        # print(f'find_sequence : {find_sequence(tokens, term)}')
        ind_term = find_sequence(tokens, term)
        # print(f'ind term : {ind_term}')
        if ind_term == -1:
            continue
        start_id, end_id = ind_term
        polarity = aspect['sentiment']

        if polarity == 'Positive':
            label[start_id] = 1  # B-POS
            label[start_id + 1:end_id] = [2] * (end_id - start_id - 1)  # I-POS
        elif polarity == 'Negative':
            label[start_id] = 3  # B-NEG
            label[start_id + 1:end_id] = [4] * (end_id - start_id - 1)  # I-NEG
        elif polarity == 'Neutral':
            label[start_id] = 5  # B-NEU
            label[start_id + 1:end_id] = [6] * (end_id - start_id - 1)  # I-NEU
    return label
labeling_ner(dataset['train'][2])

[0, 5, 6]

In [ ]:
# dataset['train']['token'][2]
# datass = dataset['train'][0]
# labeling_ner(datass)

In [ ]:
# dataset['train']['token'][3]

In [ ]:
import torch

def load_glove(path, embedding_dim=100):
    vocab_stoi = {}
    embeddings = []

    with open(path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            parts = line.strip().split()
            word = parts[0]
            vector = torch.tensor([float(x) for x in parts[1:]], dtype=torch.float)

            vocab_stoi[word] = idx
            embeddings.append(vector)

    # Add <pad> and <unk>
    vocab_stoi['<pad>'] = len(embeddings)
    embeddings.append(torch.zeros(embedding_dim))

    vocab_stoi['<unk>'] = len(embeddings)
    embeddings.append(torch.randn(embedding_dim) * 0.01)

    embedding_matrix = torch.stack(embeddings)
    return vocab_stoi, embedding_matrix

vocab_stoi, embedding_matrix = load_glove("D:/Code/Capstone/ATE/ABSA_LSTM-CRF-master/glove.6B.100d.txt")


In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class NLPDataset(Dataset):
    def __init__(self, data, labeling_ner, vocab_stoi):
        self.data = data
        self.labeling_ner = labeling_ner
        self.vocab_stoi = vocab_stoi

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        tokens = self.data['token'][idx]
        token_ids = [self.vocab_stoi.get(w, self.vocab_stoi["<unk>"]) for w in tokens]
        # token_ids = torch.tensor([self.word_to_ix[w] for w in tokens], dtype=torch.long)
        tags_ids = self.labeling_ner(self.data[idx])
        return token_ids, tags_ids

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # batch: list of (tokens, tags)|
    tokens_batch, tags_batch = zip(*batch)
    # tokens_batch = [[vocab_stoi.get(w, vocab_stoi["<unk>"]) for w in seq] for seq in tokens_batch]
    tokens_batch = [torch.tensor(t) for t in tokens_batch]
    tags_batch = [torch.tensor(t) for t in tags_batch]

    # convert each sequence to tensor
    # tokens_batch_tensors = [torch.tensor(seq, dtype=torch.long) for seq in tokens_batch]
    # tags_batch_tensors   = [torch.tensor(seq, dtype=torch.long) for seq in tags_batch]

    # pad sequences to same length
    tokens_padded = pad_sequence(tokens_batch, batch_first=True, padding_value=0)
    tags_padded   = pad_sequence(tags_batch, batch_first=True, padding_value=0)

    # create mask
    mask = tokens_padded != 0

    return tokens_padded, tags_padded, mask


In [ ]:
# from torch.utils.data import DataLoader
# # train_dataset = NLPDataset(dataset["train"],labeling_ner, vocab_stoi)
# # loader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
# i = 0
# # get first batch
# for batch in loader_train:
#     tokens_batch, tags_batch, mask = batch
#     # print("Tokens batch shape:", tokens_batch)
#     # print("Tags batch shape  :", tags_batch)
#     # print("Mask shape        :", mask)
#     if i == 3:
#         break  # just take the first batch
#     i +=1


In [ ]:
# small = dataset['train'].select(range(500))
# small[0]

In [ ]:
# ---------------------------
# Training dataset & DataLoader  .select(range(500))
# ---------------------------
small_train_data = dataset['train']
dataset_train = NLPDataset(small_train_data, labeling_ner, vocab_stoi)
loader_train = DataLoader(
    dataset_train,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn
)

# ---------------------------
# Test dataset & DataLoader .select(range(200))
# ---------------------------
small_test_data = dataset['test']
dataset_test = NLPDataset(small_test_data, labeling_ner, vocab_stoi)
loader_test = DataLoader(
    dataset_test,
    batch_size=64,
    shuffle=False,  # no shuffle for testing
    collate_fn=collate_fn
)

# # get the first batch
# batch = next(iter(train))

# # if your dataset returns (tokens, tags)
# tokens_batch, tags_batch, mask = batch

# # print first element in the batch (index 0)
# print("Tokens:", tokens_batch[3])
# print("Tags:  ", tags_batch[3])


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchcrf import CRF

class LSTM_CRF(nn.Module):
    def __init__(self, tags):
        super().__init__()
        # self.embedding = nn.Embedding(10,25)
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.lstm = nn.LSTM(100, 5, bidirectional=True, batch_first=True, dropout=0.1)
        self.fc = nn.Linear(10, tags)
        self.crf = CRF(tags, batch_first=True)

    def forward(self, x, tags=None, mask=None):
        x = self.__emissions(x)
        # print(f"Mask:\n{mask}")
        if tags == None:
            x = self.crf.decode(x)
        else:
            x = self.crf.decode(x, mask=mask)
        return x

    def __emissions(self, x):
        x = self.embedding(x)
        # print(f'emb : {x.shape}')
        x, (h, c) = self.lstm(x)
        # print(f'x:{x.shape}')
        x = self.fc(x)
        # print(f'fc:{x}')
        return x

    def loss(self, x, tags, mask=None):
        # print('strart')
        x = self.__emissions(x)
        # print(f"Tags:\n{tags}")
        # print(f"Mask:\n{mask}")
        # print(f"Tokens:\n{x}")

        x = -self.crf(x, tags, mask=mask, reduction="mean")
        return x
model = LSTM_CRF(7)

In [ ]:
# from datasets import load_dataset
# from torch.utils.data import Dataset, DataLoader
# import torch

# # ---------------------------
# # Load CoNLL-2003
# # ---------------------------
# dataset = load_dataset("lhoestq/conll2003")

# # Build vocabulary and label mapping
# all_tokens = [t for split in ["train","validation","test"] for sent in dataset[split]["tokens"] for t in sent]
# vocab = list(set(all_tokens))
# vocab_stoi = {tok:i+2 for i, tok in enumerate(vocab)}  # +2 for <pad>=0, <unk>=1
# vocab_stoi["<pad>"] = 0
# vocab_stoi["<unk>"] = 1
# vocab_itos = {i:tok for tok,i in vocab_stoi.items()}

# # Label mapping
# # labels = dataset["train"].features["ner_tags"].feature.names
# # tag_to_ix = {label:i for i,label in enumerate(labels)}
# # ix_to_tag = {i:label for label,i in tag_to_ix.items()}

# # ---------------------------
# # Dataset class
# # ---------------------------
# # class NLPDataset(Dataset):
# #     def __init__(self, hf_dataset):
# #         self.data = hf_dataset

# #     def __len__(self):
# #         return len(self.data)

# #     def __getitem__(self, idx):
# #         tokens = self.data[idx]["tokens"]
# #         token_ids = [vocab_stoi.get(t, vocab_stoi["<unk>"]) for t in tokens]
# #         tags_ids = self.data[idx]["ner_tags"]
# #         return =token_ids, dtype=torch.long), torch.tensor(tags_ids, dtype=torch.long)
# class NLPDataset(Dataset):
#     def __init__(self, data, labeling_ner, vocab_stoi):
#         self.data = data
#         self.labeling_ner = labeling_ner
#         self.vocab_stoi = vocab_stoi

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         tokens = self.data['tokens'][idx]
#         token_ids = [self.vocab_stoi.get(w, self.vocab_stoi["<unk>"]) for w in tokens]
#         # token_ids = torch.tensor([self.word_to_ix[w] for w in tokens], dtype=torch.long)
#         tags_ids = self.data[idx]["ner_tags"]
#         return token_ids, tags_ids
# # ---------------------------
# # Collate function
# # ---------------------------
# from torch.nn.utils.rnn import pad_sequence
# import torch

# def collate_fn(batch):
#     # batch: list of (tokens, tags)
#     tokens_batch, tags_batch = zip(*batch)
#     # tokens_batch = [[vocab_stoi.get(w, vocab_stoi["<unk>"]) for w in seq] for seq in tokens_batch]
#     tokens_batch = [torch.tensor(t) for t in tokens_batch]
#     tags_batch = [torch.tensor(t) for t in tags_batch]

#     # convert each sequence to tensor
#     # tokens_batch_tensors = [torch.tensor(seq, dtype=torch.long) for seq in tokens_batch]
#     # tags_batch_tensors   = [torch.tensor(seq, dtype=torch.long) for seq in tags_batch]

#     # pad sequences to same length
#     tokens_padded = pad_sequence(tokens_batch, batch_first=True, padding_value=0)
#     tags_padded   = pad_sequence(tags_batch, batch_first=True, padding_value=0)

#     # create mask
#     mask = tokens_padded != 0

#     return tokens_padded, tags_padded, mask



# # ---------------------------
# # Create DataLoaders
# # ---------------------------
# loader_train = DataLoader(NLPDataset(dataset["train"],labeling_ner, vocab_stoi), batch_size=64, shuffle=True, collate_fn=collate_fn)
# loader_val   = DataLoader(NLPDataset(dataset["validation"],labeling_ner, vocab_stoi), batch_size=64, shuffle=False, collate_fn=collate_fn)
# loader_test  = DataLoader(NLPDataset(dataset["test"],labeling_ner, vocab_stoi), batch_size=64, shuffle=False, collate_fn=collate_fn)

# print("Vocabulary size:", len(vocab_stoi))
# print("Number of tags:", len(tag_to_ix))


In [ ]:
# from torch.utils.data import DataLoader
# # train_dataset = NLPDataset(dataset["train"],labeling_ner, vocab_stoi)
# # loader_train = DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
# i = 0
# # get first batch
# for batch in loader_train:
#     tokens_batch, tags_batch, mask = batch
#     print("Tokens batch shape:", tokens_batch)
#     print("Tags batch shape  :", tags_batch)
#     print("Mask shape        :", mask)
#     if i == 3:
#         break  # just take the first batch
#     i +=1


In [ ]:
# dataset["train"][3]["tokens"]

In [ ]:
# dataset["train"][1]['ner_tags']

In [ ]:
# dataset["train"]["ner_tags"]

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
num_classes = 9  # total tags
o_class = 0      # majority "O" class
class_names = [f"Class {i}" for i in range(num_classes)]

class_weights = torch.tensor([0.00000000000000000000000000000001] + [1.0]*(num_classes-1), dtype=torch.float)
print_every = 100  # print progress every 100 batches

for epoch in range(10):
    print(f'Training Started')
    # ---------------------------
    # Training
    # ---------------------------
    all_true_train = []
    all_pred_train = []

    for batch_idx, batch in enumerate(loader_train, 1):
        tokens_batch, tags_batch, mask = batch
        mask[:, 0] = True

        optimizer.zero_grad()
        loss = model.loss(tokens_batch, tags_batch, mask)
        #* class_weights[tags_batch].mean()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            preds = model(tokens_batch, mask=mask)
            batch_size = tokens_batch.size(0)

            for i in range(batch_size):
                seq_len_i = mask[i].sum().item()
                true_tags = tags_batch[i, :seq_len_i].tolist()
                pred_tags = preds[i][:seq_len_i]
                all_true_train.extend(true_tags)
                all_pred_train.extend(pred_tags)

        if batch_idx % print_every == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}: Processed {batch_idx * batch_size} tokens so far")

    # Filter out O-class for F1
    filtered_idx = [i for i, t in enumerate(all_true_train) if t != o_class]
    filtered_true = [all_true_train[i] for i in filtered_idx]
    filtered_pred = [all_pred_train[i] for i in filtered_idx]

    train_f1_filtered = f1_score(filtered_true, filtered_pred, average='micro') if filtered_true else 0.0
    train_acc = accuracy_score(all_true_train, all_pred_train)

    per_class_acc = []
    for c in range(num_classes):
        idx = [i for i, t in enumerate(all_true_train) if t == c]
        if len(idx) == 0:
            per_class_acc.append(None)
        else:
            per_class_acc.append(sum(all_pred_train[i] == all_true_train[i] for i in idx) / len(idx))

    print(f"\n--- Epoch {epoch+1} Training Metrics ---")
    print(f"Loss: {loss.item():.4f} | Token Acc: {train_acc:.4f} | F1 (filtered): {train_f1_filtered:.4f}")
    print("Per-class Accuracy:")
    for c, acc in zip(class_names, per_class_acc):
        print(f"  {c}: {acc:.4f}" if acc is not None else f"  {c}: N/A")

    # ---------------------------
    # Testing
    # ---------------------------
    all_true_test = []
    all_pred_test = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(loader_test, 1):
            tokens_batch, tags_batch, mask = batch
            mask[:, 0] = True

            preds = model(tokens_batch, mask=mask)
            batch_size = tokens_batch.size(0)

            for i in range(batch_size):
                seq_len_i = mask[i].sum().item()
                true_tags = tags_batch[i, :seq_len_i].tolist()
                pred_tags = preds[i][:seq_len_i]
                all_true_test.extend(true_tags)
                all_pred_test.extend(pred_tags)

            if batch_idx % print_every == 0:
                print(f"Testing Epoch {epoch+1}, Batch {batch_idx}: Processed {batch_idx * batch_size} tokens so far")

    filtered_idx_test = [i for i, t in enumerate(all_true_test) if t != o_class]
    filtered_true_test = [all_true_test[i] for i in filtered_idx_test]
    filtered_pred_test = [all_pred_test[i] for i in filtered_idx_test]

    test_f1_filtered = f1_score(filtered_true_test, filtered_pred_test, average='micro') if filtered_true_test else 0.0
    test_acc = accuracy_score(all_true_test, all_pred_test)

    per_class_acc_test = []
    for c in range(num_classes):
        idx = [i for i, t in enumerate(all_true_test) if t == c]
        if len(idx) == 0:
            per_class_acc_test.append(None)
        else:
            per_class_acc_test.append(sum(all_pred_test[i] == all_true_test[i] for i in idx) / len(idx))

    print(f"\n--- Epoch {epoch+1} Testing Metrics ---")
    print(f"Token Acc: {test_acc:.4f} | F1 (filtered): {test_f1_filtered:.4f}")
    print("Per-class Accuracy:")
    for c, acc in zip(class_names, per_class_acc_test):
        print(f"  {c}: {acc:.4f}" if acc is not None else f"  {c}: N/A")
    print("\n" + "="*50)


Training Started
Epoch 1, Batch 100: Processed 6400 tokens so far
Epoch 1, Batch 200: Processed 12800 tokens so far
Epoch 1, Batch 300: Processed 19200 tokens so far
Epoch 1, Batch 400: Processed 25600 tokens so far
Epoch 1, Batch 500: Processed 32000 tokens so far
Epoch 1, Batch 600: Processed 38400 tokens so far
Epoch 1, Batch 700: Processed 44800 tokens so far
Epoch 1, Batch 800: Processed 51200 tokens so far
Epoch 1, Batch 900: Processed 57600 tokens so far
Epoch 1, Batch 1000: Processed 64000 tokens so far
Epoch 1, Batch 1100: Processed 70400 tokens so far
Epoch 1, Batch 1200: Processed 76800 tokens so far
Epoch 1, Batch 1300: Processed 83200 tokens so far
Epoch 1, Batch 1400: Processed 89600 tokens so far
Epoch 1, Batch 1500: Processed 96000 tokens so far
Epoch 1, Batch 1600: Processed 102400 tokens so far
Epoch 1, Batch 1700: Processed 108800 tokens so far

--- Epoch 1 Training Metrics ---
Loss: 6.2975 | Token Acc: 0.8899 | F1 (filtered): 0.1654
Per-class Accuracy:
  Class 0: 0.

In [ ]:
tag_to_ix = {
    "O": 0,
    "B-POS": 1,
    "I-POS": 2,
    "B-NEG": 3,
    "I-NEG": 4,
    "B-NEU": 5,
    "I-NEU": 6
}
ix_to_tag = {v: k for k, v in tag_to_ix.items()}


In [ ]:
text = "musik bagus banget aku sampai nangis."
tokens = text.split()  # simple tokenization, or use your tokenizer
token_ids = [vocab_stoi.get(t, vocab_stoi["<unk>"]) for t in tokens]
tokens_tensor = torch.tensor(token_ids).unsqueeze(0)  # add batch dimension
mask = torch.ones(tokens_tensor.shape, dtype=torch.bool)  # all real tokens
model.eval()
with torch.no_grad():
    pred_tags = model(tokens_tensor, mask=mask)  # returns list of tag indices
pred_labels = [ix_to_tag[idx] for idx in pred_tags[0]]  # pred_tags[0] because batch_size=1
for token, label in zip(tokens, pred_labels):
    print(token, label)


musik B-POS
bagus I-POS
banget B-POS
aku O
sampai O
nangis. B-POS


In [ ]:
b = [[1,2],[2,3]]
a = [[2,4], [4,5]]

b.extend(a)  # adds each sublist of a to b
print(b)


[[1, 2], [2, 3], [2, 4], [4, 5]]


In [ ]:
a = [[0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
f1_score(a, a, average='micro')

1.0

In [ ]:
# import torch

# # suppose encoded is [seq_len, batch_size] = [5, 3]
# # seq_len, batch_size = 5, 3
# # encoded = torch.randint(0, 10, (seq_len, batch_size))

# # random mask (True/False) but first token always True
# mask = torch.randint(0, 2, (seq_len, batch_size), dtype=torch.bool)
# mask[0, :] = True  # first timestep must be True for CRF

# print("Encoded:\n", encoded)
# print("Random mask:\n", mask)


In [ ]:
# import torch

# # vocab
# vocab_stoi = {
#     "<pad>": 0,
#     "<unk>": 1,
#     "John": 2,
#     "lives": 3,
#     "in": 4,
#     "New": 5,
#     "York": 6
# }

# # sentences (last two sentences have 0 to indicate padding)
# sentence = [
#     ["John", "lives", "in", "New", "York", "<pad>"],
#     ["John", "lives", "in", "York", "New", "lives"],
#     ["lives", "lives", "in", "New", "York", "<pad>"]
# ]

# # encode
# encoded = [
#     [vocab_stoi[w] if w in vocab_stoi else vocab_stoi["<unk>"] for w in sent]
#     for sent in sentence
# ]
# encoded = torch.tensor(encoded)  # shape: [batch_size, seq_len]
# print("Encoded:\n", encoded)

# # ---------------------------
# # make tags
# # ---------------------------
# tag_to_ix = {"B-PER":0, "I-PER":1, "B-LOC":2, "I-LOC":3, "O":4}

# # tags for non-padding tokens
# tags = [
#     [tag_to_ix["B-PER"], tag_to_ix["O"], tag_to_ix["O"], tag_to_ix["B-LOC"], tag_to_ix["I-LOC"], tag_to_ix["O"]],
#     [tag_to_ix["B-PER"], tag_to_ix["O"], tag_to_ix["O"], tag_to_ix["B-LOC"], tag_to_ix["I-LOC"], tag_to_ix["O"]],
#     [tag_to_ix["O"], tag_to_ix["O"], tag_to_ix["O"], tag_to_ix["B-LOC"], tag_to_ix["I-LOC"], tag_to_ix["O"]]
# ]
# tags = torch.tensor(tags)  # shape: [batch_size, seq_len]

# # ---------------------------
# # make mask: True for real tokens, False for padding
# mask = encoded != vocab_stoi["<pad>"]
# mask[0] = True
# print("Tags:\n", tags)
# print("Mask:\n", mask)


In [ ]:
print(mask[0])

tensor([True, True, True, True, True, True])


In [ ]:
# for i in range(20):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#     model = LSTM_CRF()
#     loss = model.loss(encoded, tags, mask)
#     print(loss)
#     loss.backward()
#     optimizer.step()

In [ ]:
!python -m pip install pytorch-crf

In [ ]:
import torch
import torch.nn as nn
from torchcrf import CRF
from torch.utils.data import Dataset, DataLoader

# ---------------------------
# Dummy dataset
# ---------------------------
data = [
    {"tokens": ["John", "lives", "in", "New", "York"], "tags": ["B-PER", "O", "O", "B-LOC", "I-LOC"]},
    {"tokens": ["Mary", "works", "at", "Google"], "tags": ["B-PER", "O", "O", "B-ORG"]}
]

# Build vocab and tag2idx
word_to_ix = {}
for sent in data:
    for word in sent["tokens"]:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
tag_to_ix = {"B-PER":0, "I-PER":1, "B-LOC":2, "I-LOC":3, "B-ORG":4, "I-ORG":5, "O":6}

# ---------------------------
# Dataset class
# ---------------------------
class NERDataset(Dataset):
    def __init__(self, data, word_to_ix, tag_to_ix):
        self.data = data
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        tokens = self.data[idx]["tokens"]
        tags = self.data[idx]["tags"]
        token_ids = torch.tensor([self.word_to_ix[w] for w in tokens], dtype=torch.long)
        tag_ids = torch.tensor([self.tag_to_ix[t] for t in tags], dtype=torch.long)
        return token_ids, tag_ids

dataset = NERDataset(data, word_to_ix, tag_to_ix)
loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: x)

# ---------------------------
# Simple LSTM + CRF model
# ---------------------------

# ---------------------------
# Training loop
# ---------------------------



In [ ]:
data[0]['tokens']

['John', 'lives', 'in', 'New', 'York']

In [ ]:
# get one batch from your loader
batch = next(iter(loader))  # grabs the first batch
tokens_batch, tags_batch = zip(*batch)

# pad sequences to max length in batch
max_len = max(len(t) for t in tokens_batch)
batch_size = len(batch)

tokens_padded = torch.zeros(batch_size, max_len, dtype=torch.long)
tags_padded = torch.zeros(batch_size, max_len, dtype=torch.long)
mask = torch.zeros(batch_size, max_len, dtype=torch.bool)

for i, (t, y) in enumerate(zip(tokens_batch, tags_batch)):
    tokens_padded[i, :len(t)] = t
    tags_padded[i, :len(y)] = y
    mask[i, :len(t)] = 1

# ensure first timestep is True for CRF
mask[0] = True

print("Tokens padded:\n", tokens_padded)
print("Tags padded:\n", tags_padded)
print("Mask:\n", mask)


Tokens padded:
 tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 0]])
Tags padded:
 tensor([[0, 6, 6, 2, 3],
        [0, 6, 6, 4, 0]])
Mask:
 tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True, False]])


In [ ]:
for i in range(20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model = LSTM_CRF()
    loss = model.loss(tokens_padded, tags_padded, mask)
    print(loss)
    loss.backward()
    optimizer.step()

In [ ]:
import numpy

In [ ]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased")
# outputs = model(input_ids)  # input_ids shape: (batch_size, seq_len)


OpenCV bindings requires "numpy" package.
Install it via command:
    pip install numpy


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
No module named 'numpy.core.multiarray'

In [ ]:
for i in range(20):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model = LSTM_CRF()
    loss = model.loss(encoded, tags, mask)
    print(loss)
    loss.backward()
    optimizer.step()

strart
Tags:
tensor([[0, 4, 4, 2, 3, 4],
        [0, 4, 4, 2, 3, 4],
        [4, 4, 4, 2, 3, 4]])
Mask:
tensor([[ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])
Tokens:
tensor([[[ 0.1054,  0.5158, -0.1154,  0.2473,  0.0839,  0.1271, -0.1564],
         [ 0.2386,  0.4579, -0.2578,  0.3971, -0.2138,  0.1635, -0.1007],
         [ 0.2685,  0.3010, -0.1350,  0.2836, -0.1957,  0.1078, -0.1129],
         [ 0.3029,  0.3788, -0.0396,  0.2126, -0.2045,  0.0870, -0.1658],
         [ 0.0869,  0.4848, -0.4288,  0.5283, -0.1605,  0.2429, -0.0614],
         [ 0.2137,  0.4056, -0.1671,  0.2671, -0.1420,  0.1801, -0.2401]],

        [[ 0.1322,  0.4488, -0.0644,  0.1796,  0.1584,  0.0481, -0.1683],
         [ 0.2285,  0.4319, -0.2723,  0.4005, -0.2553,  0.1600, -0.1759],
         [ 0.2781,  0.2908, -0.1002,  0.2504, -0.1608,  0.0951, -0.0953],
         [ 0.1097,  0.4581, -0.3639,  0.4544, -0.1816,  0.2448, -0.1027],
         [ 0.2675,  0.4134, -0.0754,  0.2635, -0.1886, 

ValueError: the first two dimensions of emissions and mask must match, got (3, 6) and (2, 5)